In [13]:
from helpers import *
import xgboost as xgb
from icecream import ic

In [14]:
data = np.genfromtxt('data.txt')
data = data.reshape((data.shape[0] // 21, 21, data.shape[1]))

In [15]:
for i, hand in enumerate(data):
    data[i] = normalize_hand(hand)

In [16]:
hand_sample = data[0, :, :]
np.random.shuffle(data)
data = data.T
X, y = data[:-1].T, data[-1].T
split = int(.80 * X.shape[0])
X_train, X_test = X[:split, :], X[split:, :]
y_train, y_test = y[:split], y[split:]

In [17]:
for fingers_count in range(6):
    ic(y[y == fingers_count].shape[0])

# Balance data
y_shapes = []
temp_y = y[:, 0]
unbal_indices = []
for fingers_count in range(np.unique(temp_y).shape[0]):
    unbal_indices.append(np.where(temp_y == fingers_count)[0])
    shape_ = unbal_indices[fingers_count].shape[0]
    y_shapes.append(shape_)
ic(y_shapes)
bal_indices = []
min_shape = min(y_shapes)
for fingers_count in range(len(unbal_indices)):
    bal_indices.append(unbal_indices[fingers_count][0:min_shape])
bal_indices = np.array(bal_indices).flatten()
ic(bal_indices.shape)
y_new = y[bal_indices]
X_new = X[bal_indices]
ic(X_new.shape)
ic(y_new.shape)

ic| y[y == fingers_count].shape[0]: 37653
ic| y[y == fingers_count].shape[0]: 39774
ic| y[y == fingers_count].shape[0]: 39774
ic| y[y == fingers_count].shape[0]: 39732
ic| y[y == fingers_count].shape[0]: 39606
ic| y[y == fingers_count].shape[0]: 39753
ic| y_shapes: [1793, 1894, 1894, 1892, 1886, 1893]
ic| bal_indices.shape: (10758,)
ic| X_new.shape: (10758, 21, 3)
ic| y_new.shape: (10758, 21)


(10758, 21)

In [18]:
X = X_new
y = y_new

for fingers_count in range(6):
    ic(y[y == fingers_count].shape[0])

ic| y[y == fingers_count].shape[0]: 37653
ic| y[y == fingers_count].shape[0]: 37653
ic| y[y == fingers_count].shape[0]: 37653
ic| y[y == fingers_count].shape[0]: 37653
ic| y[y == fingers_count].shape[0]: 37653
ic| y[y == fingers_count].shape[0]: 37653


In [19]:
model = xgb.XGBClassifier()

In [20]:
# Flatten for preprocessing, training and testing
X_train = X_train.reshape(X_train.shape[0] * X_train.shape[1], X_train.shape[-1])
y_train = np.ndarray.flatten(y_train)

X_test = X_test.reshape(X_test.shape[0] * X_test.shape[1], X_test.shape[-1])
y_test = np.ndarray.flatten(y_test)

In [21]:
ic(X_train.shape)
ic(X_test.shape)

ic| X_train.shape: (189021, 3)
ic| X_test.shape: (47271, 3)


(47271, 3)

In [22]:
model.fit(X_train, y_train)

[17:04:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [23]:
prediction = model.predict(X_test)
accuracy = np.average(y_test == prediction)
print('Accuracy: ', accuracy)

Accuracy:  0.8398806879482135


In [24]:
save_model(model, 'finalized_model.sav')